In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
#format for print 
np.set_printoptions(formatter={'float':"{:6.11g}".format})

mv=pd.read_csv('../BaseContent/input/tmdb_5000_movies.csv')
rating=pd.read_csv('../BaseContent/input/ratings_small - Copy.csv')

In [ ]:
# print(rating.shape)
# rating = rating[rating["movieId"] < 48665 ].reset_index(drop=True)
# print(rating.shape)
# divide train and test set
rating_train, rating_test = train_test_split(
    rating, test_size=0.15, random_state=42)

train_set = rating_train.values
test_set = rating_test.values

print(train_set.shape)
print(rating_test)

n_users = 600

In [ ]:
# Change genres column to define genres list 
from ast import literal_eval

features = ['genres']
for feature in features:
    mv[feature] = mv[feature].apply(literal_eval)
  
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    #Return empty list in case of missing/malformed data
    return []

features = ['genres']

for feature in features:
    mv[feature] = mv[feature].apply(get_list)    
mv.head(5)



In [25]:
mv.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[Action, Crime, Drama]",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
# Change genres column into others column and id movie
columns_genres = ['id', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
genres_dataframe = pd.DataFrame(columns=columns_genres)
for index in mv.index:
    value = mv['id'][index]
    tmp = [value]
    for genres in columns_genres:
           if (genres != "id"):
               if genres in mv['genres'][index]:
                   tmp.append(int(1))
               else:
                   tmp.append(int(0))
    genres_dataframe.loc[len(genres_dataframe.index)] = tmp
    
genres_dataframe.head(5)

In [26]:
genres_dataframe.head(5)

,id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,19995,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,285,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,206647,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,49026,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,49529,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X0 = genres_dataframe.values
X_train_counts = X0[:, -19:]
# print(X_train_counts)
print(X_train_counts)

In [38]:
#change to tfidf create feature vector for items
import joblib
from sklearn.feature_extraction.text import TfidfTransformer
import pickle

transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

# store features movie vectors
# joblib.dump(tfidf, 'bc_tfidf.pkl')

# test = joblib.load('bc_tfidf.pkl')

# print(test)

print(tfidf[[0,1],])

[[0.   0.47 0.55 0.   0.   0.   0.   0.   0.   0.69 0.   0.   0.   0.
  0.   0.   0.   0.   0.  ]
 [0.   0.47 0.55 0.   0.   0.   0.   0.   0.   0.69 0.   0.   0.   0.
  0.   0.   0.   0.   0.  ]]


In [ ]:
import joblib

def store_model(model, model_name=""):
    # NOTE: sklearn.joblib faster than pickle of Python
    # INFO: can store only ONE object in a file
    if model_name == "":
        model_name = type(model).__name__
    joblib.dump(model, '../BaseContent/model_users/' + model_name + '_model.pkl')


def load_model(model_name):
    # Load objects into memory
    #del model
    model = joblib.load('../BaseContent/model_users/' + model_name + '_model.pkl')
    # print(model)
    return model

In [ ]:
# get movie that rated by user
np.set_printoptions(formatter={'float': "{:6.11g}".format})


def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id

    """
    y = rate_matrix[:, 0]  # all users

    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1
    # while index in python starts from 0

    ids = np.where(y == (user_id + 1))[0]
    tmp_ids = []
    for item in ids:
        if item.size > 0:
            tmp_ids.append(item)
    
    item_ids = rate_matrix[tmp_ids,1] - 1  # index starts from 0
   
    scores = rate_matrix[tmp_ids, 2]
    # print(ids)
    return (item_ids, scores)


In [48]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1]  # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))



for n in range(n_users):
    ids, scores = get_items_rated_by_user(train_set, n)
    ids = ids.astype(int)

    clf = Ridge(alpha=0.01, fit_intercept=True)

    Xhat = tfidf[ids, :]


    clf.fit(Xhat, scores)
  
    store_model(clf, f'User{n}') #store model for each user 
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_
  
    
print(W)
print(b.shape)

# Y = np.asarray(scores_Larges)
# X = np.asarray(np.matrix(Xhat_Large))
# print(np.matrix(Xhat_Large))
# clfLarge = Ridge(alpha=0.01, fit_intercept=True)
# clfLarge.fit(X, Y)
# store_model(clfLarge)


[[ 0.    0.    0.   ...  0.    0.    0.  ]
 [-1.66 -0.79 -0.43 ...  0.34 -2.   -0.16]
 [-1.22  0.15 -0.24 ...  0.14 -0.05  3.15]
 ...
 [ 0.54  0.05  1.25 ... -0.09  1.16  1.3 ]
 [ 0.   -0.66  1.39 ...  0.49  0.    0.  ]
 [ 0.    0.    0.   ... -0.58  0.33  0.  ]]
(1, 600)


In [50]:
# Predict 
Yhat = tfidf.dot(W) + b
display(Yhat.shape)
print(Yhat)

(4803, 600)

[[2.48 3.56 3.41 ... 3.7  2.32 3.91]
 [2.48 3.56 3.41 ... 3.7  2.32 3.91]
 [1.52 3.61 3.46 ... 3.53 1.69 4.04]
 ...
 [2.87 3.23 3.29 ... 3.61 3.89 1.13]
 [2.91 3.92 3.81 ... 3.45 3.98 2.26]
 [2.91 4.   3.81 ... 3.32 3.98 2.26]]


In [53]:
n = 10
np.set_printoptions(precision=2) 
ids, scores = get_items_rated_by_user(train_set, n)
ids = ids.astype(int)
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [3423  777  125 2595 2041  784 1407   69 1026 2761  168  922 1200]
True ratings     : [3.  4.5 4.  4.5 3.5 3.5 5.  1.  4.5 3.  3.  5.  5. ]
Predicted ratings: [3.51 4.86 4.3  4.5  3.09 3.53 4.7  1.12 4.49 3.2  3.03 4.86 4.3 ]


In [51]:
from math import sqrt
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        ids = ids.astype(int)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

In [52]:
print('RMSE for training:', evaluate(Yhat, train_set, W, b))
print('RMSE for test    :', evaluate(Yhat, test_set, W, b))


RMSE for training: 0.9014677841520674
RMSE for test    : 1.077380149870499


In [ ]:
# Test load model for api

model = load_model(f"User{1}")

# tmp  = [
#     [0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0],
#     [0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
#     [0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
#     [0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0],
#     [0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0]
# ]

# columns_genres_test_load = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
#                   'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
# genres_dataframe_test_load = pd.DataFrame( tmp,columns=columns_genres_test_load)

# print(genres_dataframe_test_load)
# X1 = genres_dataframe_test_load.values

# <=> yHjat = tfidf.dot(W) + b
yhat = model.predict(tfidf[[665], :])

yhat = model.predict(tfidf[ids, :])

print(yhat.shape)
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', yhat)